In [121]:
import pandas  as pd
import math
import numpy  as np
import matplotlib.pyplot as plt
import csv
import random as r
from sklearn import datasets
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)


In [122]:
tmp = datasets.load_iris()
data_iris = tmp.data
target = tmp.target
train_data, test_data, train_label, test_label = train_test_split(data_iris, target, test_size=0.1, random_state=69)
# print(train_data)  # [[5.7 3.  4.2 1.2], [5.7 2.6 3.5 1. ], [7.2 3.  5.8 1.6]]
# print(test_data)  # [[5.7 3.  4.2 1.2], [5.7 2.6 3.5 1. ], [7.2 3.  5.8 1.6]]
# print(train_label) # == [2 2 1 1 1 0 0 1 0 1 2 0 2 0 1 2 2 2 1 2 0 1 2 0 0 1 1 2 0 0]
# print(test_label) # == [2 2 1 1 1 0 0 1 0 1 2 0 2 0 1 2 2 2 1 2 0 1 2 0 0 1 1 2 0 0]

In [123]:
class LogisticRegression:        
    def sigmoid(z): 
        return 1 / (1 + np.exp(-z))
    def loss(y, y_hat):
        loss = -np.mean(y*(np.log(y_hat)) - (1-y)*np.log(1-y_hat))
        return loss
    def gradients(data, y, y_hat):
        num_points = data.shape[0] # num_points-> number of training examples.
        dw = (1/num_points)*np.dot(data.T, (y_hat - y))      
        return dw
    def normalize(X): #return list ;without nan
        num_points, n = np.matrix(X).shape
        for _ in range(n):
            with np.errstate(divide='ignore', invalid='ignore'):
                X = (X - X.mean(axis=0))/X.std(axis=0)
        X = np.nan_to_num(X,nan = 1)
        return (X).tolist()
    def train_LR(train_data,train_labels,alpha = 0.002, max_iter = 1000, debug = False):
        Train_data = np.array(train_data)                               ## num_points*n = datapoint*features*n
        normalized_data = LogisticRegression.normalize(Train_data)      # num_points*n = datapoint*features

        num_points = len(normalized_data)                                #no. of data points
        n = len(normalized_data[0])                                      # no. of features
        slope = (([0.69 for _ in range(n)]))    #r.random()        ## 1 * features ;randomly intialize slope == {b0, b1 ...}
        count = 0
        while(count < max_iter):
            slope = (np.matrix(slope).T)
            y_hat_all = (np.dot(slope.T,np.matrix(np.matrix(normalized_data).T)).tolist())[0]
            y_hats = [0 for _ in range(num_points)]
            for i in range(0,num_points):
                y_hats[i] = LogisticRegression.sigmoid(y_hat_all[i])

            error = (np.matrix(y_hats) - np.matrix(train_labels)).tolist() # 1*num_points

            p = (np.dot(np.matrix(error).T,np.matrix(normalized_data[i])))
            sumed = np.sum(p,axis=0)
            sumed_list = (sumed.tolist())

            change = (alpha/num_points)*np.array(sumed_list)
            slope = (slope).T
            slope -= change
            slope = slope.tolist()[0]
            count+=1
        return slope
    def finding_accuracy(confidence_list,testing_labels, threshold,debug = False):
        j=1
        predicted_labels = [j := 0 if i < threshold else 1 for i in confidence_list]
        ratio = (sum([1 for i in range(len(predicted_labels)) if predicted_labels[i] == testing_labels[i]]))/len(predicted_labels) , 
        if(debug == True) : return ratio
        accuracy = max([(sum([1 for i in range(len(predicted_labels)) if predicted_labels[i] == testing_labels[i]]))/len(predicted_labels) , 
                    (sum([1 for i in range(len(predicted_labels)) if predicted_labels[i] == testing_labels[i]]))/len(predicted_labels)])
        
        return accuracy
    def test_LR(test_data, test_labels, slope,threshold = 0.5,debug = False):
        test_data = LogisticRegression.normalize(np.array(test_data))   #converting test_data in same form
        x = (np.dot(np.matrix(slope),np.matrix(test_data).T)).tolist()[0]
        prob = [LogisticRegression.sigmoid(i) for i in x]

        accuracy = LogisticRegression.finding_accuracy(prob,test_labels,threshold,debug=False)
        return accuracy, prob
    def Logistic_Regression(train_features,train_labels,test_features,test_labels,accuracy_only=True,debug = False):
        slope = LogisticRegression.train_LR(train_features,train_labels)
        accuracy, prob_list = LogisticRegression.test_LR(test_features,test_labels,slope)
        if(accuracy_only) : return accuracy
        return prob_list , accuracy

We can run Logistic regression 3 times <br /> 
With label as {0,(1 or 2)} , {1,(0 or 1)} ,{2,(1 or 0)}

In [124]:
train_label_s =  [  [0 if x == 0 else 1 for x in train_label],
                    [1 if x == 1 else 0 for x in train_label],
                    [1 if x == 2 else 0 for x in train_label]
                    ]

test_label_s =  [   [0 if x == 0 else 1 for x in test_label],
                    [1 if x == 1 else 0 for x in test_label],
                    [1 if x == 2 else 0 for x in test_label]
                    ]
prob = [[],[],[]]
# print(train_label_s[0])
# print(train_label_s[1])
# print(train_label_s[2])
# print(test_label_s[0])
# print(test_label_s[1])
# print(test_label_s[2])
# print(test_label_s)
for i in range(3):
    prob[i], acc = LogisticRegression.Logistic_Regression(train_data,train_label_s[i],test_data,test_label_s[i],accuracy_only=False)
    # print(acc)

prob = (np.array(prob)).T
prob = prob.tolist()
labels = []
# print(prob)
labels = [i.index(min(i))for i in prob]
accuracy = (sum([1 for i in range(len(labels)) if labels[i] == test_label[i]]))/len(labels)
print(f"""\nAccuracy with without preprocessing and classification by Logistic Regression = {accuracy}\n""")


Accuracy with without preprocessing and classification by Logistic Regression = 0.8



#### Does `Logistic Reg in multiclass` provide better results than the ones reported in `LDA then kNN` ?<br />
No, kNN works very fine (for the given data), <br /> whereas Logistic on mutiple struggle a bit (here)